Primeira parte:
    • Representação (genótipo): string de bits #OK
    • Recombinação: “cut-and-crossfill” crossover
    • Probabilidade de Recombinação: 90%
    • Mutação: troca de genes #OK
    • Probabilidade de Mutação: 40%
    • Seleção de pais: ranking - Melhor de 2 de 5 escolhidos aleatoriamente #OK
    • Seleção de sobreviventes: substituição do pior
    • Tamanho da população: 100 #OK
    • Número de filhos gerados: 2
    • Inicialização: aleatória #OK
    • Condição de término: Encontrar a solução, ou 10.000 avaliações de fitness
    • Fitness #OK

In [182]:
from random import randint
import numpy as np

In [183]:
populationSize = 100

### Funções auxiliares

In [184]:
def binaryToDecimal(binary): 
    binary1 = binary
    decimal, i, n = 0, 0, 0
    while(binary != 0): 
        dec = binary % 10
        decimal = decimal + dec * pow(2, i) 
        binary = binary//10
        i += 1
    return decimal

In [206]:
def sortPopulation(population):
    # Ordena uma população pelo seu valor de fitness
    population.sort(key=lambda x:x['fitnessValue'])
    return population

### Criar população

In [185]:
def generateChess():
    aux = []
    for i in range(0,8):
        aux.append(generateQueen())
    return aux

In [186]:
def generateQueen():
    queenPosition = ''
    for bitPosition in range(1,4):
        queenPosition += str(randint(0,1))
    return queenPosition

In [187]:
def criarPopulacao():
    aux = []
    for populationMember in range(populationSize):
        aux.append({'individual': generateChess() , 'fitnessValue': -1 }) 
        #Cria um objeto composto do individuo e seu valor de fitness    
    return aux

### Quantidade de 'choques'

In [188]:
# Se duas rainhas estiverem na mesma coluna
def columnThreaten(individual):
    threatenColumn = 0
    for column in range(0,8):
        binaryColumn = np.binary_repr(column, width=3)
        if individual.count(binaryColumn) >= 2:
            threatenColumn += 1
    return threatenColumn

In [189]:
# Se duas rainhas estiverem na mesma diagonal
def diagonalThreaten(individual):
    deltaRow = 0
    deltaCol = 0
    threatenDiagonal = 0
    for x in range(0,8):
        for y in range(0,8):
            if x != y:
                deltaRow = abs(x-y)
                deltaCol = abs(binaryToDecimal(int(individual[x])) - binaryToDecimal(int(individual[y])))
                if (deltaRow == deltaCol):
                    threatenDiagonal += 1
    return threatenDiagonal/2

### Calculo do fitness

In [190]:
def fitnessIndividual(individual):
    value = 1 / (1 +(columnThreaten(individual) + diagonalThreaten(individual)))
    return value

In [191]:
def fitness(population):
    fitnessValue = []
    for individual in population:
        individual['fitnessValue'] = fitnessIndividual(individual['individual'])

#### O valor do fitness será dado pela função 1/1+qntdChoques(tabuleiro)
#### Queremos aumentar esse valor de fitness

### Mutação

Escolher um individuo(tabuleiro) aleatório e trocar a posição de uma rainha

In [192]:
def mutationSwap(individual):
    randomQueen = randint(0,8)
    individual[randomQueen] = generateQueen()

### Seleção de pais

In [193]:
def chooseRandomIndividuals(population):
    aux = []
    for i in range(0,5):
        value = randint(0,populationSize)
        aux.append(population[value])
    return aux

In [207]:
def selectParents(population):
    parents = chooseRandomIndividuals(population) #Array de objetos onde os elementos são os individuos e o valor do fitness
    parents = sortPopulation(parents) # Ordenando os pais de acordo com o fitnessValue
    return (parents[-2], parents[-1]) #Retornando os dois ultimos elementos da lista que são os com melhores fitness

### Seleção dos sobreviventes

#### Remover os dois menos adaptados da população

In [215]:
def selectLastElements(population):
    sortedPopulation = sortPopulation(population)
    ### Trocarei os dois primeiros elementos pelos dois novos filhos
    ### sortedPopulation[0], sortedPopulation[1] = doisFilhosGerados 